In [92]:
#%pip install git+https://github.com/cair/tmu.git
%pip install tmu
#%pip install numpy==1.26.4


[notice] A new release of pip is available: 24.3 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [93]:
import numpy as np
import pandas as pd
import pickle
import os

from tmu.models.autoencoder.autoencoder import TMAutoEncoder

In [94]:
# Create folder models if it does not exist
os.makedirs("models", exist_ok=True)

In [95]:
def load_train_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)

    print(f"Event: {event_id}, Shape: {X.shape}")
    return X

In [96]:
def save_model(tm: TMAutoEncoder, filename: str):
    a, d = tm.X_train, tm.encoded_X_train

    tm.X_train = None
    tm.encoded_X_train = None

    with open(f"./models/{filename}", "wb") as f:
        pickle.dump(tm, f)

    tm.X_train = a
    tm.encoded_X_train = d

In [97]:
wind_farm = "C"

available_datasets_df = pd.read_csv(f"../../../data/care_to_compare/Wind Farm {wind_farm}/event_info.csv",
                                    delimiter=';')

available_datasets = available_datasets_df["event_id"].values

print(available_datasets)

# Skip 36

train_datasets = available_datasets[available_datasets != 36]
test_datasets = available_datasets[available_datasets != 36]

# Load all datasets into one array
X_train = [load_train_dataset(wind_farm, i) for i in train_datasets]

# Combine all items of X_train
X_train = np.concatenate(X_train)

print(f"X_train shape: {X_train.shape}")

[55 81 47 12  4 18 28 39 66 15 78 79 30 33 11 44 49 31 67  9 91  5 90 70
 35 16 76  8 85  6 62 36 56 94 54 43 50 64 46 65 61 93 75 41 58 48 88 57
 32 89 59 63 80 37 29  1 20 60]
Event: 55, Shape: (4000, 1190)
Event: 81, Shape: (4000, 1190)
Event: 47, Shape: (4000, 1190)
Event: 12, Shape: (4000, 1190)
Event: 4, Shape: (4000, 1190)
Event: 18, Shape: (4000, 1190)
Event: 28, Shape: (4000, 1190)
Event: 39, Shape: (4000, 1190)
Event: 66, Shape: (4000, 1190)
Event: 15, Shape: (4000, 1190)
Event: 78, Shape: (4000, 1190)
Event: 79, Shape: (4000, 1190)
Event: 30, Shape: (4000, 1190)
Event: 33, Shape: (4000, 1190)
Event: 11, Shape: (4000, 1190)
Event: 44, Shape: (4000, 1190)
Event: 49, Shape: (4000, 1190)
Event: 31, Shape: (4000, 1190)
Event: 67, Shape: (4000, 1190)
Event: 9, Shape: (4000, 1190)
Event: 91, Shape: (4000, 1190)
Event: 5, Shape: (4000, 1190)
Event: 90, Shape: (4000, 1190)
Event: 70, Shape: (4000, 1190)
Event: 35, Shape: (4000, 1190)
Event: 16, Shape: (4000, 1190)
Event: 76, Shape: (

In [98]:
def train(args):
    tm = TMAutoEncoder(
        number_of_clauses=args["num_clauses"],
        T=args["T"],
        s=args["s"],
        output_active=args["output_active"],
        max_included_literals=args["max_included_literals"],
        accumulation=args["accumulation"],
        feature_negation=args["feature_negation"],
        platform=args["platform"],
        output_balancing=args["output_balancing"],
    )

    print(f"Starting training for {args['epochs']} epochs")

    for e in range(args["epochs"]):
        tm.fit(X_train, number_of_examples=args["number_of_examples"])

        save_model(tm, f"latest_{e}.pkl")
        print(f"Epoch: {e + 1}")


In [99]:
number_of_features = X_train.shape[1]
output_active = np.arange(number_of_features, dtype=np.uint32)
number_of_clauses = int(4 * number_of_features)

print(f"Number of features {number_of_features}, Number of clauses: {number_of_clauses}")

Number of features 1190, Number of clauses: 4760


In [100]:
args: dict = {
    "clause_weight_threshold": 1,
    "number_of_examples": 10,
    "output_active": output_active,
    "accumulation": 1,
    "num_clauses": number_of_clauses,
    "T": int(number_of_clauses * 0.75 * 100),
    "s": 25,
    "epochs": 25,
    "platform": "CPU",
    "output_balancing": 0,
    "max_included_literals": number_of_features,
    "feature_negation": True,
}

result = train(args)

Starting training for 25 epochs
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
